In [188]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer,MinMaxScaler
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold,StratifiedKFold,cross_val_score,GridSearchCV,train_test_split

In [2]:
def read_files(Train,Test):
    titanic_train=pd.read_csv(Train,index_col=0)
    titanic_test=pd.read_csv(Test,index_col=0)
    return titanic_train,titanic_test

In [3]:
def fix_names(df_for_names):
    Titles_dict={'Mrs.':'Mrs.','Miss.':'Miss.','Master.':'Master.','Lady.':'Lady.','Dr.':'Sir.','Sir.':'Sir.','Major.':'Sir.',
             'Capt.':'Sir.','Col.':'Sir.','Mr.':'Mr.','Rev.':'Sir.','Don.':'Sir.','Countess.':'Lady.',
             'Mlle.':'Miss.','Ms.':'Miss.','Mme.':'Mr.','Jonkheer.':'Mr.','Dona.':'Miss.'}
    for lab,row in df_for_names.iterrows():
        words=row['Name'].split()
        for word in words:
            if word in Titles_dict.keys():
                df_for_names.loc[lab,'Name']=Titles_dict[word]
    return df_for_names

In [4]:
def fix_tickets(df_tickets):
    Ticket_dict={'1':1,'2':2,'3':3,'A':4,'C':5,'P':6,'S':7}
    for lab,row in df_tickets.iterrows():
        if row['Ticket'][0] in Ticket_dict.keys():
            df_tickets.loc[lab,'Ticket']=Ticket_dict[row['Ticket'][0]]
        else:
            df_tickets.loc[lab,'Ticket']=0     
    return  df_tickets

In [5]:
def fix_cabin_with_similarities(df_cabin):
    for lab, row in df_cabin.iterrows():
        if pd.notna(row['Cabin']):
            df_cabin.loc[lab,'Cabin']=row['Cabin'][0]
    Cabin_df=df_cabin.groupby(['Cabin']).mean()
    Cabin_labels=list(Cabin_df.index)
    Cabin_Fare=Cabin_df[['Fare']]
    Cabin_Parch=Cabin_df[['Parch']]
    Cabin_Fare_np=Cabin_Fare.to_numpy()
    Cabin_Parch_np=Cabin_Parch.to_numpy()
    Cabin_Fare_Kmeans=KMeans(n_clusters=7).fit(Cabin_Fare_np)
    Cabin_Parch_Kmeans=KMeans(n_clusters=7).fit(Cabin_Parch_np)
    Cabin_Fare_Labels=list(Cabin_Fare_Kmeans.labels_)
    Cabin_Parch_Labels=list(Cabin_Parch_Kmeans.labels_)
    Cabin_Parch_dict=dict(zip(Cabin_Parch_Labels,Cabin_labels))
    Cabin_Fare_dict=dict(zip(Cabin_Fare_Labels,Cabin_labels))
    for lab, row in df_cabin.iterrows():
        if pd.isna(row['Cabin']):
            if pd.notna(row['Fare']):
                point=row[['Fare']].to_numpy()
                CB_key=Cabin_Fare_Kmeans.predict([point])[0]
                CB_Value=Cabin_Fare_dict[CB_key]
                df_cabin.loc[lab,'Cabin']=CB_Value
            else:
                point=row[['Parch']].to_numpy()
                CB_key=Cabin_Parch_Kmeans.predict([point])[0]
                CB_Value=Cabin_Parch_dict[CB_key]
                df_cabin.loc[lab,'Cabin']=CB_Value
    return df_cabin

In [6]:
def fix_cabin_with_most_frequent(df_cabin):
    for lab, row in df_cabin.iterrows():
        if pd.notna(row['Cabin']):
            df_cabin.loc[lab,'Cabin']=row['Cabin'][0]
    Max_Frequency=df_cabin['Cabin'].value_counts().idxmax()
    for lab, row in df_cabin.iterrows():
        if pd.isna(row['Cabin']):
            df_cabin.loc[lab,'Cabin']=Max_Frequency
    return df_cabin

In [7]:
def fix_age(df_age):
    for lab,row in df_age.iterrows():
        if pd.notna(row['Age']):
            if row['Age']<7:
                df_age.loc[lab,'Age']=0
            elif row['Age']>55:
                df_age.loc[lab,'Age']=2
            else:
                df_age.loc[lab,'Age']=1
        else:
            df_age.loc[lab,'Age']=3
    return df_age

In [8]:
def fix_age_median(df_age):
    Title_Age_Table=df_age.groupby(['Name']).median()[['Age']]
    Count=0
    for lab,row in df_age.iterrows():
        if pd.isna(row['Age']):
            label=row['Name']
            df_age.loc[lab,'Age']=Title_Age_Table.loc[label,'Age']
    df_age.loc[:,'Age'].fillna(df_age['Age'].mean(),inplace=True)
    return df_age

In [9]:
def model_maker(optimizer='adam',initializer='glorot_uniform'):
    model=Sequential()
    model.add(Dense(units=8,activation='relu',kernel_initializer=initializer,input_shape=(28,)))
    #model.add(Dense(units=n_neurons2,activation='relu'))
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'],verbose=0)
    return model

In [10]:
titanic_train,titanic_test=read_files('train.csv','test.csv')
X0=fix_tickets(titanic_train)
T0=fix_tickets(titanic_test)
X1=fix_names(X0)
T1=fix_names(T0)
X2=fix_cabin_with_most_frequent(X1)
T2=fix_cabin_with_most_frequent(T1)
X3=fix_age_median(X2)
T3=fix_age_median(T2)

In [11]:
Parch_dict={0:0,1:1,2:1,3:2,4:2,5:2,6:2,9:2}
X3.loc[:,'Parch'].replace(Parch_dict,inplace=True)
T3.loc[:,'Parch'].replace(Parch_dict,inplace=True)

In [12]:
SibSP_dict={0:0,1:1,2:2,3:3,4:3,5:3,8:2}
X3.loc[:,'SibSp'].replace(SibSP_dict,inplace=True)
T3.loc[:,'SibSp'].replace(SibSP_dict,inplace=True)

In [13]:
Embark_list=X3.loc[:,'Embarked'].value_counts()[:].index.tolist()
print(Embark_list)

['S', 'C', 'Q']


In [14]:
X3.loc[:,'Embarked'].fillna(Embark_list[0],inplace=True)
T3.loc[:,'Embarked'].fillna(Embark_list[0],inplace=True)

In [15]:
X3.isna().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [26]:
Train=X3.iloc[:,1:]
Test=T3
y=X3[['Survived']].to_numpy()

In [17]:
cs = MinMaxScaler()

In [18]:
Titanic_Columns=list(Train.columns)
print(Titanic_Columns)

['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [19]:
ContinousParameters=['Age','Fare']

In [20]:
CategoricalParameters=[member for member in Titanic_Columns if (member not in ContinousParameters and member!='Survived' and member!='Parch' and member!='Ticket')]

In [21]:
TrainContinuous = cs.fit_transform(Train[ContinousParameters])
TestContinuous = cs.transform(Test[ContinousParameters])
One_Hot = OneHotEncoder().fit(Train[CategoricalParameters])
TrainCategorical = One_Hot.transform(Train[CategoricalParameters]).toarray()
TestCategorical = One_Hot.transform(Test[CategoricalParameters]).toarray()

In [22]:
TTrainX = np.hstack((TrainCategorical, TrainContinuous))
TTestX = np.hstack((TestCategorical, TestContinuous))

In [190]:
model=Sequential()
model.add(Dropout(0.15, input_shape=(28,)))
model.add(Dense(units=9,activation='relu'))
#model.add(Dense(units=6,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'],verbose=0)
NN_model=model.fit(TTrainX,y,epochs=200,batch_size=10)

Train on 891 samples
Epoch 1/200
891/891 [==============================] - 1s 816us/sample - loss: 0.6864 - accuracy: 0.5241
Epoch 2/200
891/891 [==============================] - 0s 162us/sample - loss: 0.6179 - accuracy: 0.7318
Epoch 3/200
891/891 [==============================] - 0s 155us/sample - loss: 0.5576 - accuracy: 0.7722
Epoch 4/200
891/891 [==============================] - 0s 135us/sample - loss: 0.5134 - accuracy: 0.7744
Epoch 5/200
891/891 [==============================] - 0s 130us/sample - loss: 0.5046 - accuracy: 0.7755
Epoch 6/200
891/891 [==============================] - 0s 138us/sample - loss: 0.4841 - accuracy: 0.7688
Epoch 7/200
891/891 [==============================] - 0s 140us/sample - loss: 0.4697 - accuracy: 0.7924
Epoch 8/200
891/891 [==============================] - 0s 144us/sample - loss: 0.4646 - accuracy: 0.8036
Epoch 9/200
891/891 [==============================] - 0s 136us/sample - loss: 0.4629 - accuracy: 0.7834
Epoch 10/200
891/891 [============

Epoch 77/200
891/891 [==============================] - 0s 134us/sample - loss: 0.4138 - accuracy: 0.8204
Epoch 78/200
891/891 [==============================] - 0s 141us/sample - loss: 0.4134 - accuracy: 0.8171
Epoch 79/200
891/891 [==============================] - 0s 187us/sample - loss: 0.4120 - accuracy: 0.8193
Epoch 80/200
891/891 [==============================] - 0s 139us/sample - loss: 0.4138 - accuracy: 0.8159
Epoch 81/200
891/891 [==============================] - 0s 162us/sample - loss: 0.4137 - accuracy: 0.8227
Epoch 82/200
891/891 [==============================] - 0s 184us/sample - loss: 0.4271 - accuracy: 0.8159
Epoch 83/200
891/891 [==============================] - 0s 144us/sample - loss: 0.4178 - accuracy: 0.8126
Epoch 84/200
891/891 [==============================] - 0s 147us/sample - loss: 0.4193 - accuracy: 0.8148
Epoch 85/200
891/891 [==============================] - 0s 131us/sample - loss: 0.4121 - accuracy: 0.8171
Epoch 86/200
891/891 [========================

In [175]:
Titanic_prediction_NN=model.predict_classes(TTestX)

In [176]:
TTestX.shape

(418, 28)

In [73]:
NN_8Nodes_100Epochs_20Batches=Titanic_prediction_NN

In [78]:
NN_6Nodes_100Epochs_20Batches=Titanic_prediction_NN

In [91]:
NN_4Nodes_100Epochs_20Batches=Titanic_prediction_NN

In [103]:
NN_5Nodes_100Epochs_20Batches=Titanic_prediction_NN

In [117]:
NN_8Nodes_10Epochs_20Batches=Titanic_prediction_NN

In [134]:
NN_7Nodes_10Epochs_20Batches=Titanic_prediction_NN

In [148]:
NN_9Nodes_10Epochs_20Batches=Titanic_prediction_NN

In [162]:
NN_10Nodes_10Epochs_20Batches=Titanic_prediction_NN

In [177]:
NN_9Nodes_20Epochs_20Batches=Titanic_prediction_NN

In [163]:
print((NN_6Nodes_100Epochs_20Batches==NN_10Nodes_10Epochs_20Batches).sum(),(NN_8Nodes_100Epochs_20Batches==NN_10Nodes_10Epochs_20Batches).sum())

387 392


In [164]:
print((NN_4Nodes_100Epochs_20Batches==NN_10Nodes_10Epochs_20Batches).sum(),(NN_5Nodes_100Epochs_20Batches==NN_10Nodes_10Epochs_20Batches).sum())

383 387


In [165]:
print((NN_8Nodes_10Epochs_20Batches==NN_10Nodes_10Epochs_20Batches).sum(),(NN_7Nodes_10Epochs_20Batches==NN_10Nodes_10Epochs_20Batches).sum())

386 386


In [179]:
print((NN_9Nodes_10Epochs_20Batches==NN_9Nodes_20Epochs_20Batches).sum(),(NN_10Nodes_10Epochs_20Batches==NN_9Nodes_20Epochs_20Batches).sum())

392 387


In [180]:
print(Titanic_prediction_NN)

[[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]


In [181]:
Titanic_prediction_NN.shape

(418, 1)

In [182]:
T3['Survived']=Titanic_prediction_NN

In [183]:
titanic_test.head(10)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
892,3,Mr.,male,34.5,0,0,3,7.8292,C,Q,0
893,3,Mrs.,female,47.0,1,0,3,7.0000,C,S,0
894,2,Mr.,male,62.0,0,0,2,9.6875,C,Q,0
895,3,Mr.,male,27.0,0,0,3,8.6625,C,S,0
896,3,Mrs.,female,22.0,1,1,3,12.2875,C,S,1
897,3,Mr.,male,14.0,0,0,0,9.2250,C,S,0
898,3,Miss.,female,30.0,0,0,3,7.6292,C,Q,1
899,2,Mr.,male,26.0,1,1,2,29.0000,C,S,0
900,3,Mrs.,female,18.0,0,0,2,7.2292,C,C,1


In [184]:
prediction_df=T3[['Survived']]

In [185]:
prediction_df.to_csv('Downloads/Datascience/KaggleStuff/NN_prediction.csv')